# Gait Phase Feature Analysis

This notebook analyzes the EMG features distribution across the 3 gait phase classes:
- **Stance** (Class 0)
- **Swing** (Class 1)
- **None** (Class 2)

It also visualizes feature evolution over the continuous gait cycle.

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Add src to path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from lib.data_loader import Enabl3sDataLoader
from lib.preprocess import EMGPreprocessor
from lib.features import extract_statistical_features
from lib.dataset import MultiModeDataset

# Plot styling
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_context("notebook", font_scale=1.2)

In [2]:
PHASE_NAMES = {
    0: "Stance",
    1: "Swing"
}

DYNAMIC_MODES = [1, 2, 3]

EMG_CHANNELS = ['TA', 'MG']
LOAD_CHANNELS = ['TA', 'MG', 'Mode', 'Ankle_Angle', 'Knee_Angle']
TARGET_FS = 250

In [3]:
def load_data(subject, data_root="../../data"):
    print(f"Loading {subject}...")
    loader = Enabl3sDataLoader(data_root, subject, target_fs=TARGET_FS)
    raw_dir = os.path.join(data_root, subject, 'Raw')
    
    circuit_files = [f for f in os.listdir(raw_dir) if f.endswith('_raw.csv')]
    circuits_to_load = range(1, len(circuit_files) + 1)
    
    df = loader.load_dataset_batch(circuits_to_load, LOAD_CHANNELS)
    
    if df.empty:
        return df
        
    # Filter modes
    df = df[df['Mode'].isin(DYNAMIC_MODES)]
    
    # Preprocess
    preprocessor = EMGPreprocessor()
    df[EMG_CHANNELS] = preprocessor.apply_filter(df[EMG_CHANNELS].values)
    df[EMG_CHANNELS] = preprocessor.rectify(df[EMG_CHANNELS].values)
    
    is_dynamic = df['Mode'].isin(DYNAMIC_MODES)
    df.loc[is_dynamic & (df['Label_Phase'] == 1), 'Phase_Class'] = 0
    df.loc[is_dynamic & (df['Label_Phase'] == 0), 'Phase_Class'] = 1
    
    df['Phase_Name'] = df['Phase_Class'].map(PHASE_NAMES)
    
    return df

def extract_features_from_window(window, channels):
    """Extract MAV, RMS, and WL features from each EMG channel."""
    features = {}
    threshold = 0.005

    for i, ch in enumerate(channels):
        channel_data = window[:, i]
        features[f'{ch}_MAV'] = np.mean(np.abs(channel_data))
        features[f'{ch}_RMS'] = np.sqrt(np.mean(channel_data ** 2))
        features[f'{ch}_WL'] = np.sum(np.abs(np.diff(channel_data)))
    return features

def create_feature_dataset(df, emg_channels, window_size_ms, step_size_ms, target_fs):
    """Create windowed dataset and return a DataFrame with features."""
    dataset = MultiModeDataset(
        df=df,
        feature_cols=emg_channels,
        label_col='Label_Phase',
        window_size_ms=window_size_ms,
        step_size_ms=step_size_ms,
        fs=target_fs
    )
    
    print(f"Extracting features from {len(dataset)} windows...")
    data_list = []
    
    for i, (window, label) in enumerate(dataset):
        feats = extract_features_from_window(window, emg_channels)
        feats['Mode'] = int(label)
        feats['Phase_Name'] = PHASE_NAMES.get(int(label), 'Unknown')
        data_list.append(feats)
        
    return pd.DataFrame(data_list)

In [4]:
# Load Data
df = load_data("AB156")
print(f"Loaded {len(df)} samples")
df.head()

11:49:00 - INFO - Starting batch load: Circuits 1 to 50...


Loading AB156...


11:49:01 - INFO - Labels generated via Processed Events (Heel Contact + Toe Off). Swing Ratio: 57.07%
11:49:02 - INFO - Labels generated via Processed Events (Heel Contact + Toe Off). Swing Ratio: 33.32%
11:49:02 - INFO - Labels generated via Processed Events (Heel Contact + Toe Off). Swing Ratio: 33.05%
11:49:03 - INFO - Labels generated via Processed Events (Heel Contact + Toe Off). Swing Ratio: 40.71%
11:49:03 - INFO - Labels generated via Processed Events (Heel Contact + Toe Off). Swing Ratio: 26.70%
11:49:04 - INFO - Labels generated via Processed Events (Heel Contact + Toe Off). Swing Ratio: 30.75%
11:49:04 - INFO - Labels generated via Processed Events (Heel Contact + Toe Off). Swing Ratio: 33.10%
11:49:05 - INFO - Labels generated via Processed Events (Heel Contact + Toe Off). Swing Ratio: 30.31%
11:49:05 - INFO - Labels generated via Processed Events (Heel Contact + Toe Off). Swing Ratio: 52.06%
11:49:07 - INFO - Labels generated via Processed Events (Heel Contact + Toe Off). 

Loaded 158545 samples


,TA,MG,Ankle_Angle,Knee_Angle,Mode,Ankle_Norm,Circuit_ID,Label_Phase,Phase_Class,Phase_Name
1072,0.043589,0.025710,2.140002,3.099489,3.0,0.438784,1.0,0.0,1.0,Swing
1755,0.057657,0.019377,2.049668,2.053330,1.0,0.435925,1.0,0.0,1.0,Swing
1756,0.019647,0.051852,2.050889,2.053330,1.0,0.435964,1.0,0.0,1.0,Swing
1757,0.010063,0.076189,2.050889,2.052110,1.0,0.435964,1.0,0.0,1.0,Swing
1758,0.009555,0.015933,2.050889,2.053330,1.0,0.435964,1.0,0.0,1.0,Swing


## 1. Class Distribution

In [ ]:
plt.figure(figsize=(8, 4))
sns.countplot(data=df, x='Phase_Name', order=['Stance', 'Swing'])
plt.title("Sample Count by Phase")
plt.show()

## 2. Feature Distributions

In [6]:
feat_dataset = create_feature_dataset(df, EMG_CHANNELS, window_size_ms=200, step_size_ms=50, target_fs=TARGET_FS)
feat_dataset.head()

Extracting features from 9967 windows...


,TA_MAV,TA_RMS,TA_WL,MG_MAV,MG_RMS,MG_WL,Mode,Phase_Name
0,0.011810,0.019309,0.479248,0.020979,0.028238,0.826006,0,Stance
1,0.009892,0.016129,0.387404,0.019496,0.028863,0.713467,0,Stance
2,0.006981,0.008981,0.293538,0.032581,0.046005,0.991096,0,Stance
3,0.006928,0.009058,0.285873,0.028905,0.042907,0.963725,0,Stance
4,0.008179,0.011488,0.375474,0.035674,0.049056,1.186978,0,Stance


In [ ]:
plt.figure(figsize=(10, 5))
sns.countplot(data=feat_dataset, x='Phase_Name', order=list(PHASE_NAMES.values()))
plt.title("Distribution of Samples per Phase")
plt.xlabel("Phase")
plt.ylabel("Count")
plt.show()

## 4. Feature Distributions by Class

Here we visualize the distribution of each feature (MAV, RMS, WL) for each Muscle (TA, MG, RF). Distinct distributions imply good separability.

In [ ]:
feature_cols = [c for c in feat_dataset.columns if c not in ['Mode', 'Phase_Name']]

plt.figure(figsize=(20, 15))
for i, col in enumerate(feature_cols):
    
    plt.subplot(3, 3, i + 1)
    sns.boxplot(data=feat_dataset, x='Phase_Name', y=col, showfliers=False, order=list(PHASE_NAMES.values()))
    plt.title(f"{col} Distribution")
    plt.xticks(rotation=45)
    plt.ylabel("Value")
    plt.xlabel("")
    
plt.tight_layout()
plt.show()

## 5. Dimensionality Reduction

We use PCA to project the 9-dimensional feature space (3 muscles * 3 features) into 2D to visualize global clusters.

In [ ]:
# Scale features
X = feat_dataset[feature_cols].values
y = feat_dataset['Phase_Name'].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

pca_df = pd.DataFrame(data=X_pca, columns=['PC1', 'PC2'])
pca_df['Mode'] = y

plt.figure(figsize=(12, 8))
sns.scatterplot(data=pca_df, x='PC1', y='PC2', hue='Mode', alpha=0.6, palette='viridis')
plt.title(f"PCA of EMG Features (Explained Variance: {pca.explained_variance_ratio_.sum():.2f})")
plt.show()

## Train the model

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

clf = LinearDiscriminantAnalysis()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print("Classification Report:")
print(classification_report(y_test, y_pred))

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred, labels=list(PHASE_NAMES.values()))
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=list(PHASE_NAMES.values()), 
            yticklabels=list(PHASE_NAMES.values()))
plt.ylabel('True Class')
plt.xlabel('Predicted Class')
plt.title('Confusion Matrix')
plt.show()

Classification Report:
              precision    recall  f1-score   support

      Stance       0.80      0.87      0.83      1445
       Swing       0.87      0.80      0.83      1546

    accuracy                           0.83      2991
   macro avg       0.83      0.83      0.83      2991
weighted avg       0.84      0.83      0.83      2991

